# Preprocessing

In [1]:
# Essential Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning & Statistics
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy.stats import skew, kurtosis
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Deep Learning (TensorFlow/Keras)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Google Drive Mounting (For Colab Users)
from google.colab import drive
drive.mount('/content/drive')

# Visualization Settings
%matplotlib inline
sns.set(color_codes=True)

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/My Drive/CAMEL_ML_internal/camel_ml_df.csv")

df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

#CNN-LSTM

**Train/Test Procedures**   

Below are three train/test procedures:  

1. **All gauges (without gauge ID as input):**  
   - The model is trained on a subset of gauge stations (e.g., 70% of all stations) together and tested on another unseen subset of gauge stations.  
   - This approach helps evaluate the model's **spatial generalization capability**.  

2. **Each site separately:**  
   - The model is trained on individual sites from a subset of all gauge stations.  
   - It is then tested on another subset, with each site handled separately.  

3. **Train/test within sites:**  
   - The model is **trained and tested within each site** independently.  


##All gauges (without gauge id)

In [ ]:
# Get unique gauge_ids
unique_gauge_ids = df['gauge_id'].unique()

# Define the number of samples for each set
num_train = 470
num_dev = 68
num_test = 136

# Randomly sample unique gauge_ids for train, dev, and test sets
train_gauge_ids = np.random.choice(unique_gauge_ids, num_train, replace=False)
remaining_gauge_ids = np.setdiff1d(unique_gauge_ids, train_gauge_ids)
dev_gauge_ids = np.random.choice(remaining_gauge_ids, num_dev, replace=False)
test_gauge_ids = np.setdiff1d(remaining_gauge_ids, dev_gauge_ids)

# Create the subsets
train_df = df[df['gauge_id'].isin(train_gauge_ids)]
dev_df = df[df['gauge_id'].isin(dev_gauge_ids)]
test_df = df[df['gauge_id'].isin(test_gauge_ids)]

# Define features and target
features = ['dayl', 'prcp', 'srad', 'swe', 'tmax', 'tmin', 'vp']
target = ['streamflow']

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import numpy as np

# Define a function to train on multiple sites and test on others
def train_on_multiple_sites_cnn_lstm(train_df, test_df, train_gauge_ids_sample, test_gauge_ids_sample, timesteps=20, epochs=40, batch_size=32):
    # Filter the dataset for the selected training sites
    train_df_sample = train_df[train_df['gauge_id'].isin(train_gauge_ids_sample)]

    # Filter the dataset for the selected testing sites
    test_df_sample = test_df[test_df['gauge_id'].isin(test_gauge_ids_sample)]

    # Initialize the StandardScaler and scale the features
    scaler_X = StandardScaler()
    X_train_scaled = scaler_X.fit_transform(train_df_sample[features])
    X_test_scaled = scaler_X.transform(test_df_sample[features])

    # Prepare the target variables
    y_train = train_df_sample[target].values.ravel()
    y_test = test_df_sample[target].values.ravel()

    # Check if there are enough samples for CNN-LSTM to work
    if len(X_train_scaled) < timesteps or len(X_test_scaled) < timesteps:
        raise ValueError(f"Not enough samples for CNN-LSTM with timesteps={timesteps}.")

    # Define the number of features
    features_count = X_train_scaled.shape[1]

    # Reshape the training data for CNN-LSTM
    n_samples_train = X_train_scaled.shape[0]
    n_samples_test = X_test_scaled.shape[0]

    X_train_reshaped = np.zeros((n_samples_train - timesteps + 1, timesteps, features_count))
    X_test_reshaped = np.zeros((n_samples_test - timesteps + 1, timesteps, features_count))

    for i in range(timesteps, n_samples_train + 1):
        X_train_reshaped[i - timesteps] = X_train_scaled[i - timesteps:i]

    for i in range(timesteps, n_samples_test + 1):
        X_test_reshaped[i - timesteps] = X_test_scaled[i - timesteps:i]

    # Reshape the target variables accordingly
    y_train_reshaped = y_train[timesteps-1:]
    y_test_reshaped = y_test[timesteps-1:]

    # Build the CNN-LSTM model
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(timesteps, features_count)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=64, activation='relu'))
    model.add(Dense(units=32, activation='linear'))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Define early stopping criteria
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

    # Train the CNN-LSTM model with early stopping
    history = model.fit(X_train_reshaped, y_train_reshaped,
                        epochs=epochs,
                        batch_size=batch_size,
                        verbose=1,
                        validation_split=0.1,
                        callbacks=[early_stopping])

    # Make predictions on the train set
    y_train_pred = model.predict(X_train_reshaped)
    # Compute R² score for the train set
    r2_train = r2_score(y_train_reshaped, y_train_pred)

    # Make predictions on the test set
    y_test_pred = model.predict(X_test_reshaped)
    # Compute R² score for the test set
    r2_test = r2_score(y_test_reshaped, y_test_pred)

    print(f'Training R² score: {r2_train:.4f}')
    print(f'Testing R² score: {r2_test:.4f}')

    return r2_train, r2_test

# Example usage:
# Manually select sample of gauge stations for training from train_df
train_gauge_ids_sample = train_df['gauge_id'].unique()[:5]  # Manually select 5 gauges for training

# Manually select sample of gauge stations for testing from test_df
test_gauge_ids_sample = test_df['gauge_id'].unique()[:5]  # Manually select 5 gauges for testing

# Run the training and testing process
r2_train, r2_test = train_on_multiple_sites_cnn_lstm(train_df, test_df, train_gauge_ids_sample, test_gauge_ids_sample)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/40
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 4394555.0000 - val_loss: 1895056.1250
Epoch 2/40
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3528288.5000 - val_loss: 2151550.2500
Epoch 3/40
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 3155888.7500 - val_loss: 2365231.7500
Epoch 4/40
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 3112291.5000 - val_loss: 1839626.0000
Epoch 5/40
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 2972540.2500 - val_loss: 1754518.8750
Epoch 6/40
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 2978971.5000 - val_loss: 1340814.2500
Epoch 7/40
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2979355.5000 - val_loss: 2052393.3750
Epoch 8/40
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 2852259.7500 - val_loss: 1578994.7500
Epoch 9/40
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 2728358.2500 - val_loss: 1531672.6250
Epoch 10/40
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 2610696.7500 - val_loss: 2158168.7500

##Each site separately

In [ ]:
# Get unique gauge_ids
unique_gauge_ids = df['gauge_id'].unique()

# Define the number of samples for each set
num_train = 470
num_dev = 68
num_test = 136

# Randomly sample unique gauge_ids for train, dev, and test sets
train_gauge_ids = np.random.choice(unique_gauge_ids, num_train, replace=False)
remaining_gauge_ids = np.setdiff1d(unique_gauge_ids, train_gauge_ids)
dev_gauge_ids = np.random.choice(remaining_gauge_ids, num_dev, replace=False)
test_gauge_ids = np.setdiff1d(remaining_gauge_ids, dev_gauge_ids)

# Create the subsets
train_df = df[df['gauge_id'].isin(train_gauge_ids)]
dev_df = df[df['gauge_id'].isin(dev_gauge_ids)]
test_df = df[df['gauge_id'].isin(test_gauge_ids)]

# Define features and target
features = ['dayl', 'prcp', 'srad', 'swe', 'tmax', 'tmin', 'vp']
target = ['streamflow']

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd

# Define features and target
features = ['dayl', 'prcp', 'srad', 'swe', 'tmax', 'tmin', 'vp']
target = ['streamflow']

# Initialize the model training function
def train_cnn_lstm_on_sites(train_df, train_gauge_ids_sample, timesteps=20, epochs=40, batch_size=32):
    # Initialize dictionary to store R² scores for each training gauge
    train_r2_scores = {}

    # Filter the dataset for the selected training sites
    for gauge_id in train_gauge_ids_sample:
        train_df_sample = train_df[train_df['gauge_id'] == gauge_id]

        # Initialize the StandardScaler and scale the features
        scaler_X = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(train_df_sample[features])

        # Prepare the target variables
        y_train = train_df_sample[target].values.ravel()

        # Define the number of features
        features_count = X_train_scaled.shape[1]

        # Reshape the training data for CNN-LSTM
        n_samples_train = X_train_scaled.shape[0]
        X_train_reshaped = np.zeros((n_samples_train - timesteps + 1, timesteps, features_count))

        for i in range(timesteps, n_samples_train + 1):
            X_train_reshaped[i - timesteps] = X_train_scaled[i - timesteps:i]

        # Reshape the target variable accordingly
        y_train_reshaped = y_train[timesteps-1:]

        # Build the CNN-LSTM model
        model = Sequential()
        model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(timesteps, features_count)))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Flatten())
        model.add(Dense(units=64, activation='relu'))
        model.add(Dense(units=32, activation='linear'))
        model.add(Dense(units=1))

        # Compile the model
        model.compile(optimizer='adam', loss='mean_squared_error')

        # Define early stopping criteria
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

        # Train the CNN-LSTM model with early stopping
        history = model.fit(X_train_reshaped, y_train_reshaped,
                            epochs=epochs,
                            batch_size=batch_size,
                            verbose=1,
                            validation_split=0.1,
                            callbacks=[early_stopping])

        # Make predictions on the train set
        y_train_pred = model.predict(X_train_reshaped)
        # Compute R² score for the train set
        train_r2_scores[gauge_id] = r2_score(y_train_reshaped, y_train_pred)

        print(f'Site {gauge_id}:')
        print(f'  Train R² score: {train_r2_scores[gauge_id]:.4f}')
        print('-' * 50)

    # Return the trained model and the training R² scores
    return model, train_r2_scores

### Training Code Block (on train_df) ###
train_gauge_ids_sample = train_df['gauge_id'].unique()[:5]  # Manually specify a batch of training gauges
trained_model, train_r2_scores = train_cnn_lstm_on_sites(train_df, train_gauge_ids_sample)  # Train the model

### Testing Code Block (on test_df) ###
def process_gauges_for_testing(test_gauge_ids, df, features, target, trained_model, timesteps=20):
    # Initialize dictionary to store R² scores for each gauge
    test_r2_scores = {}

    # Process each gauge in the selected test_gauge_ids
    for gauge_id in test_gauge_ids:
        # Filter data for the current gauge
        test_site_data = df[df['gauge_id'] == gauge_id]

        # Initialize the StandardScaler and scale the features
        scaler_X = StandardScaler()
        X_test_scaled = scaler_X.fit_transform(test_site_data[features])

        # Prepare the target variables
        y_test = test_site_data[target].values.ravel()

        # Define the number of features
        features_count = X_test_scaled.shape[1]

        # Reshape the testing data for CNN-LSTM
        n_samples_test = X_test_scaled.shape[0]
        X_test_reshaped = np.zeros((n_samples_test - timesteps + 1, timesteps, features_count))

        for i in range(timesteps, n_samples_test + 1):
            X_test_reshaped[i - timesteps] = X_test_scaled[i - timesteps:i]

        # Reshape the target variable accordingly
        y_test_reshaped = y_test[timesteps-1:]

        # Make predictions on the test set using the trained model
        y_test_pred = trained_model.predict(X_test_reshaped)

        # Compute R² score for the test set
        test_r2_scores[gauge_id] = r2_score(y_test_reshaped, y_test_pred)

        print(f'Site {gauge_id}:')
        print(f'  Test R² score: {test_r2_scores[gauge_id]:.4f}')
        print('-' * 50)

    return test_r2_scores

# Manually specify a batch of gauge_ids to test on (select first 5 gauges from test_df)
test_gauge_ids_sample = test_df['gauge_id'].unique()[:5]

# Test the model on the selected gauges
test_r2_scores = process_gauges_for_testing(test_gauge_ids_sample, test_df, features, target, trained_model=trained_model)

# Create DataFrames for both training and testing R² scores
train_r2_df = pd.DataFrame(list(train_r2_scores.items()), columns=['gauge_id', 'train_r2'])
test_r2_df = pd.DataFrame(list(test_r2_scores.items()), columns=['gauge_id', 'test_r2'])

# Print the resulting DataFrames
print("Training R² scores:")
print(train_r2_df)
print("\nTest R² scores:")
print(test_r2_df)

Epoch 1/40


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 13734165.0000 - val_loss: 6299408.5000
Epoch 2/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 5928606.5000 - val_loss: 5202194.5000
Epoch 3/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4788137.0000 - val_loss: 4618447.5000
Epoch 4/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4579385.0000 - val_loss: 4044331.5000
Epoch 5/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4194849.0000 - val_loss: 3317159.7500
Epoch 6/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3580169.0000 - val_loss: 2914406.2500
Epoch 7/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3323081.5000 - val_loss: 2671880.5000
Epoch 8/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2881557.7500 - val_loss: 2569176.7500
Epoch 9/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2887769.0000 - val_loss: 2591484.7500
Epoch 10/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2825345.0000 - val_loss: 2457438.0000
Epoch 11/40
357/357 ━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


359/359 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1803994.7500 - val_loss: 747557.8125
Epoch 2/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 894469.4375 - val_loss: 566397.3750
Epoch 3/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 738044.6875 - val_loss: 499882.5625
Epoch 4/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 621509.6250 - val_loss: 479044.5625
Epoch 5/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 501858.0000 - val_loss: 450584.3438
Epoch 6/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 607435.4375 - val_loss: 434965.9062
Epoch 7/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 536067.1250 - val_loss: 434879.5938
Epoch 8/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 497454.9375 - val_loss: 423097.6875
Epoch 9/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 431667.4062 - val_loss: 418940.7500
Epoch 10/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 411662.4688 - val_loss: 435849.9375
Epoch 11/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/s

...

359/359 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 361361.9688 - val_loss: 154821.6406
Epoch 2/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 227933.7656 - val_loss: 121204.7500
Epoch 3/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 159504.5781 - val_loss: 117762.2188
Epoch 4/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 132273.1719 - val_loss: 109462.8047
Epoch 5/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 117292.6562 - val_loss: 99760.2344
Epoch 6/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 110993.0000 - val_loss: 101049.2734
Epoch 7/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 115714.5234 - val_loss: 92931.1328
Epoch 8/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 103053.0625 - val_loss: 89204.8828
Epoch 9/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 99819.0312 - val_loss: 91173.5469
Epoch 10/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 83845.1172 - val_loss: 89856.5859
Epoch 11/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - lo

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


359/359 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 145756.2188 - val_loss: 91046.2031
Epoch 2/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 89302.1719 - val_loss: 72891.3672
Epoch 3/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 74138.2578 - val_loss: 61235.4375
Epoch 4/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 63881.3672 - val_loss: 56771.5781
Epoch 5/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 56127.8125 - val_loss: 61444.0078
Epoch 6/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 59108.2500 - val_loss: 54929.7734
Epoch 7/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 49260.7070 - val_loss: 66185.0547
Epoch 8/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 58279.7422 - val_loss: 57281.7305
Epoch 9/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 59930.5078 - val_loss: 55084.8750
Epoch 10/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 48501.5898 - val_loss: 54175.8633
Epoch 11/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 46113.20

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


359/359 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 174878.2344 - val_loss: 119669.2891
Epoch 2/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 97398.5781 - val_loss: 100367.4766
Epoch 3/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 75573.8906 - val_loss: 83298.7266
Epoch 4/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 59698.9492 - val_loss: 82961.4922
Epoch 5/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 58068.4219 - val_loss: 85855.8438
Epoch 6/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 66203.7266 - val_loss: 80050.3359
Epoch 7/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 54066.6445 - val_loss: 83980.4453
Epoch 8/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 55455.5195 - val_loss: 81668.4688
Epoch 9/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 56653.0000 - val_loss: 86971.7656
Epoch 10/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 59536.4805 - val_loss: 84231.5078
Epoch 11/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 56845.

##Train/test within sites

In [ ]:
# Function to calculate NSE and PBIAS
def calculate_nse(y_true, y_pred):
    return 1 - np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2)

def calculate_pbias(y_true, y_pred):
    return 100 * np.sum(y_true - y_pred) / np.sum(y_true)

# Function to process a batch of gauge stations and run the CNN-LSTM model
def process_gauges_individually(df, gauge_ids_batch):
    metrics = []

    for gauge_id in gauge_ids_batch:
        site_data = df[df['gauge_id'] == gauge_id]
        split_index = int(len(site_data) * 0.8)

        train_site_data = site_data.iloc[:split_index]
        test_site_data = site_data.iloc[split_index:]

        scaler_X = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(train_site_data[features])
        X_test_scaled = scaler_X.transform(test_site_data[features])

        y_train = train_site_data[target].values.ravel()
        y_test = test_site_data[target].values.ravel()

        timesteps = 20
        features_count = X_train_scaled.shape[1]

        n_samples_train = X_train_scaled.shape[0]
        n_samples_test = X_test_scaled.shape[0]

        X_train_reshaped = np.zeros((n_samples_train - timesteps + 1, timesteps, features_count))
        X_test_reshaped = np.zeros((n_samples_test - timesteps + 1, timesteps, features_count))

        for i in range(timesteps, n_samples_train + 1):
            X_train_reshaped[i - timesteps] = X_train_scaled[i - timesteps:i]

        for i in range(timesteps, n_samples_test + 1):
            X_test_reshaped[i - timesteps] = X_test_scaled[i - timesteps:i]

        y_train_reshaped = y_train[timesteps-1:]
        y_test_reshaped = y_test[timesteps-1:]

        # Build the Simplified CNN-LSTM Model
        model = Sequential()
        model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(timesteps, features_count)))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Flatten())
        model.add(Dense(units=32, activation='relu'))
        model.add(Dense(units=1))

        model.compile(optimizer='adam', loss='mean_squared_error')

        early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

        model.fit(X_train_reshaped, y_train_reshaped, epochs=40, batch_size=32, verbose=1, validation_split=0.1, callbacks=[early_stopping])

        y_train_pred = model.predict(X_train_reshaped).flatten()
        y_test_pred = model.predict(X_test_reshaped).flatten()

        train_r2 = r2_score(y_train_reshaped, y_train_pred)
        test_r2 = r2_score(y_test_reshaped, y_test_pred)
        train_mse = mean_squared_error(y_train_reshaped, y_train_pred)
        test_mse = mean_squared_error(y_test_reshaped, y_test_pred)
        train_rmse = np.sqrt(train_mse)
        test_rmse = np.sqrt(test_mse)
        train_mae = mean_absolute_error(y_train_reshaped, y_train_pred)
        test_mae = mean_absolute_error(y_test_reshaped, y_test_pred)
        train_nse = calculate_nse(y_train_reshaped, y_train_pred)
        test_nse = calculate_nse(y_test_reshaped, y_test_pred)
        train_pbias = calculate_pbias(y_train_reshaped, y_train_pred)
        test_pbias = calculate_pbias(y_test_reshaped, y_test_pred)

        metrics.append({
            'gauge_id': gauge_id,
            'train_r2': train_r2, 'test_r2': test_r2,
            'train_rmse': train_rmse, 'test_rmse': test_rmse,
            'train_mse': train_mse, 'test_mse': test_mse,
            'train_mae': train_mae, 'test_mae': test_mae,
            'train_nse': train_nse, 'test_nse': test_nse,
            'train_pbias': train_pbias, 'test_pbias': test_pbias
        })

        print(f"Gauge ID: {gauge_id} processed successfully.")

    return pd.DataFrame(metrics)

# Function to process gauge stations in chunks
def process_gauges_in_chunks(df, start_idx=0, chunk_size=30, results_file='results.csv'):
    try:
        results_df = pd.read_csv(results_file)
    except FileNotFoundError:
        results_df = pd.DataFrame(columns=['gauge_id', 'train_r2', 'test_r2', 'train_rmse', 'test_rmse',
                                           'train_mse', 'test_mse', 'train_mae', 'test_mae', 'train_nse',
                                           'test_nse', 'train_pbias', 'test_pbias'])

    unique_gauge_ids = df['gauge_id'].unique()
    total_gauges = len(unique_gauge_ids)

    gauge_ids_batch = unique_gauge_ids[start_idx:start_idx + chunk_size]
    print(f"Processing gauge stations {start_idx + 1} to {min(start_idx + chunk_size, total_gauges)}...")

    df_subset = df[df['gauge_id'].isin(gauge_ids_batch)]
    metrics_df = process_gauges_individually(df_subset, gauge_ids_batch)

    results_df = pd.concat([results_df, metrics_df], ignore_index=True)
    results_df.to_csv(results_file, index=False)

    print("Results saved to", results_file)

# Example usage:
features = ['dayl', 'prcp', 'srad', 'swe', 'tmax', 'tmin', 'vp']
target = ['streamflow']

process_gauges_in_chunks(df, start_idx=630, chunk_size=50)

Processing gauge stations 631 to 674...
Epoch 1/40


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


287/287 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 167.7730 - val_loss: 40.3711
Epoch 2/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 81.7611 - val_loss: 31.9982
Epoch 3/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 68.5802 - val_loss: 28.7734
Epoch 4/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 60.5967 - val_loss: 34.1978
Epoch 5/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 63.2516 - val_loss: 30.8152
Epoch 6/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 47.0066 - val_loss: 29.4564
Epoch 7/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 47.5147 - val_loss: 29.1929
Epoch 8/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 44.6180 - val_loss: 30.5529
Epoch 8: early stopping
Restoring model weights from the end of the best epoch: 3.
319/319 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Gauge ID: 13018300 processed successfully.
Epoch 1/40


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


288/288 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 684712.6875 - val_loss: 146703.7344
Epoch 2/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 237415.2656 - val_loss: 93755.2266
Epoch 3/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 181349.8438 - val_loss: 81413.1953
Epoch 4/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 158084.4219 - val_loss: 72562.3203
Epoch 5/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 156782.2500 - val_loss: 66035.5781
Epoch 6/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 140252.3125 - val_loss: 60282.1914
Epoch 7/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 129732.1016 - val_loss: 56613.7617
Epoch 8/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 118523.7578 - val_loss: 53625.3398
Epoch 9/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 109003.9297 - val_loss: 54058.4961
Epoch 10/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 108976.6172 - val_loss: 49774.2930
Epoch 11/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss

...

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


259/259 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 160.8114 - val_loss: 24.7517
Epoch 2/40
259/259 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 145.3108 - val_loss: 27.6246
Epoch 3/40
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 110.7548 - val_loss: 24.9578
Epoch 4/40
259/259 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 94.5437 - val_loss: 22.8762
Epoch 5/40
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 103.7396 - val_loss: 16.3391
Epoch 6/40
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 136.8862 - val_loss: 18.9933
Epoch 7/40
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 72.4928 - val_loss: 20.5860
Epoch 8/40
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 90.7716 - val_loss: 14.1517
Epoch 9/40
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 50.0311 - val_loss: 28.7900
Epoch 10/40
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 73.0396 - val_loss: 12.4711
Epoch 11/40
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 71.8139 - val_loss: 14.3663
Epoch 12/40
259/259 ━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


288/288 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 18536476.0000 - val_loss: 13565604.0000
Epoch 2/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 8508763.0000 - val_loss: 10979330.0000
Epoch 3/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 9474759.0000 - val_loss: 9350323.0000
Epoch 4/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 7146437.5000 - val_loss: 7993462.0000
Epoch 5/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 5987457.5000 - val_loss: 7137225.0000
Epoch 6/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4788183.5000 - val_loss: 6428638.5000
Epoch 7/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4922403.5000 - val_loss: 5866790.0000
Epoch 8/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4705303.5000 - val_loss: 5435256.0000
Epoch 9/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 3923697.5000 - val_loss: 5104803.5000
Epoch 10/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 4102000.0000 - val_loss: 4861854.0000
Epoch 11/40
288/288 ━━━━━━━

<ipython-input-10-5c730c872401>:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, metrics_df], ignore_index=True)


In [ ]:
!pip install tensorflow numpy pandas scikit-learn

# Re-import necessary libraries since execution state was reset
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Generate synthetic time series data (5 years of daily data)
days = pd.date_range(start="2019-01-01", end="2023-12-31", freq='D')
num_samples = len(days)

# Generate inputs X1 and X2 with seasonal patterns
X1 = np.sin(np.linspace(0, 10 * np.pi, num_samples)) + np.random.normal(scale=0.1, size=num_samples)
X2 = np.cos(np.linspace(0, 10 * np.pi, num_samples)) + np.random.normal(scale=0.1, size=num_samples)

# Generate target y as a function of X1, X2 with noise
y = 0.5 * X1 + 0.3 * X2 + np.random.normal(scale=0.1, size=num_samples)

# Convert to DataFrame
data = pd.DataFrame({'Date': days, 'X1': X1, 'X2': X2, 'y': y})

# Split into train and test (last 2 months for testing)
train_size = num_samples - 60
train_data, test_data = data.iloc[:train_size], data.iloc[train_size:]

# Scale data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

train_X = scaler_X.fit_transform(train_data[['X1', 'X2']])
train_y = scaler_y.fit_transform(train_data[['y']])
test_X = scaler_X.transform(test_data[['X1', 'X2']])
test_y = scaler_y.transform(test_data[['y']])

# Convert to PyTorch tensors
train_X_tensor = torch.tensor(train_X, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
train_y_tensor = torch.tensor(train_y, dtype=torch.float32)
test_X_tensor = torch.tensor(test_X, dtype=torch.float32).unsqueeze(1)
test_y_tensor = torch.tensor(test_y, dtype=torch.float32)

# Create DataLoader
batch_size = 32
train_dataset = TensorDataset(train_X_tensor, train_y_tensor)
test_dataset = TensorDataset(test_X_tensor, test_y_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define Temporal Convolutional Network (TCN)
class TCN(nn.Module):
    def __init__(self, input_channels, output_size, num_filters=16, kernel_size=3):
        super(TCN, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, num_filters, kernel_size, padding=1)
        self.conv2 = nn.Conv1d(num_filters, num_filters, kernel_size, padding=1)
        self.fc = nn.Linear(num_filters, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = x.mean(dim=2)  # Global average pooling
        x = self.fc(x)
        return x

# Initialize model
model = TCN(input_channels=1, output_size=1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train model
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    if epoch % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.6f}")

# Evaluate model
model.eval()
with torch.no_grad():
    y_pred_test = model(test_X_tensor).numpy()
    y_true_test = test_y_tensor.numpy()

# Convert predictions back to original scale
y_pred_test_rescaled = scaler_y.inverse_transform(y_pred_test)
y_true_test_rescaled = scaler_y.inverse_transform(y_true_test)

# Compute performance metrics
mse = mean_squared_error(y_true_test_rescaled, y_pred_test_rescaled)
r2 = r2_score(y_true_test_rescaled, y_pred_test_rescaled)

mse, r2


##All gauges (with gauges ids as input)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd

# Filter the dataset to include only the first 10 unique gauge_ids
first_10_gauge_ids = df['gauge_id'].unique()[:10]
df_filtered = df[df['gauge_id'].isin(first_10_gauge_ids)]

# Convert gauge_id to categorical for the filtered data
df_filtered['gauge_id'] = df_filtered['gauge_id'].astype('category')

# Split the filtered data into train and test sets based on gauge_id
train_gauge_ids = df_filtered['gauge_id'].unique()[:8]  # First 8 sites for training
test_gauge_ids = df_filtered['gauge_id'].unique()[8:]   # Remaining 2 sites for testing

train_df = df_filtered[df_filtered['gauge_id'].isin(train_gauge_ids)]
test_df = df_filtered[df_filtered['gauge_id'].isin(test_gauge_ids)]

# Define features and target
features = ['dayl', 'prcp', 'srad', 'swe', 'tmax', 'tmin', 'vp']
target = ['streamflow']

# Prepare the X and y datasets for train and test
X_train = train_df[features + ['gauge_id']]
y_train = train_df[target]

X_test = test_df[features + ['gauge_id']]
y_test = test_df[target]

# Get all unique categories in the entire dataset for 'gauge_id'
all_gauge_ids = df_filtered['gauge_id'].unique()

# Define the ColumnTransformer with predefined categories for gauge_id
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), features),
        ('cat', OneHotEncoder(categories=[all_gauge_ids], drop='first'), ['gauge_id'])
    ])

# Apply the transformations to the training and test sets
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Define the number of timesteps and features
timesteps = 20  # Number of previous data points to consider
num_features = X_train_transformed.shape[1]  # Number of features after transformation

# Reshape the training and test data for CNN-LSTM
n_samples_train = X_train_transformed.shape[0]
n_samples_test = X_test_transformed.shape[0]

X_train_reshaped = np.zeros((n_samples_train - timesteps + 1, timesteps, num_features))
X_test_reshaped = np.zeros((n_samples_test - timesteps + 1, timesteps, num_features))

for i in range(timesteps, n_samples_train + 1):
    X_train_reshaped[i - timesteps] = X_train_transformed[i - timesteps:i]

for i in range(timesteps, n_samples_test + 1):
    X_test_reshaped[i - timesteps] = X_test_transformed[i - timesteps:i]

# Reshape the target variable accordingly
y_train_reshaped = y_train[timesteps-1:].values.ravel()
y_test_reshaped = y_test[timesteps-1:].values.ravel()

# Build the CNN-LSTM model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(timesteps, num_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='linear'))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Train the CNN-LSTM model with early stopping
history = model.fit(X_train_reshaped, y_train_reshaped,
                    epochs=40,
                    batch_size=32,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

# Make predictions on the train set
y_train_pred = model.predict(X_train_reshaped)
# Compute R² score for the train set
r2_train = r2_score(y_train_reshaped, y_train_pred)

# Make predictions on the test set
y_test_pred = model.predict(X_test_reshaped)
# Compute R² score for the test set
r2_test = r2_score(y_test_reshaped, y_test_pred)

print(f'The train R² score is: {r2_train:.4f}')
print(f'The test R² score is: {r2_test:.4f}')

# Pad zeros to match the original time series dimensions for the train data
y_train_pred_padded = np.concatenate((np.zeros(timesteps-1), y_train_pred.flatten()))
# Compute R² score for the padded train data
r2_train_padded = r2_score(y_train, y_train_pred_padded)
print(f'The R² score for the padded train data is: {r2_train_padded:.4f}')

# Pad zeros to match the original time series dimensions for the test data
y_test_pred_padded = np.concatenate((np.zeros(timesteps-1), y_test_pred.flatten()))
# Compute R² score for the padded test data
r2_test_padded = r2_score(y_test, y_test_pred_padded)
print(f'The R² score for the padded test data is: {r2_test_padded:.4f}')

<ipython-input-9-4b7defff066a>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['gauge_id'] = df_filtered['gauge_id'].astype('category')
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/40
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 1875576.1250 - val_loss: 135620.1094
Epoch 2/40
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 789037.3750 - val_loss: 108895.8125
Epoch 3/40
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 660841.8125 - val_loss: 101699.3594
Epoch 4/40
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 627990.0000 - val_loss: 97448.7578
Epoch 5/40
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 579492.1250 - val_loss: 93494.5234
Epoch 6/40
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 530678.2500 - val_loss: 95155.8906
Epoch 7/40
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 510653.8125 - val_loss: 96910.4688
Epoch 8/40
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 490776.9375 - val_loss: 95768.0781
Epoch 9/40
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 478480.8438 - val_loss: 96132.2031
Epoch 10/40
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 438852.2812 - val_loss: 98612.5859
Epoch 10: early stopping
